In [ ]:
class GAN():
    def __init__(self):
        self.img_rows = 96
        self.img_cols = 128
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    def read_image_local(self):
        images = []
        img = os.getcwd() + "/gan/test2"
        pics = os.listdir(img)
        size = 128,128
        for pic in pics:
            if 'png' in pic:
                pic_path = img + "/" + pic
                print(pic_path)
                image = Image.open(pic_path).convert("LA")
                image.thumbnail(size)

                image.save(img + '/thumbnail/'+ pic )
                image_gray = np.array(image)[:,:,0]
                print(np.shape(image_gray))
                images.append(image_gray)
        return images

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()

        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)
    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
#         (X_train, _), (_, _) = mnist.load_data()
        X_train = self.read_image_local()

        # Rescale -1 to 1
#         X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = [i/255 for i in X_train]
        
        
        
        X_train = np.expand_dims(X_train, axis=3)
        print(np.shape(X_train[0]))
        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("gan/test/mnist_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, save_interval=200)


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_10 (Flatten)         (None, 12288)             0         
_________________________________________________________________
dense_64 (Dense)             (None, 512)               6291968   
_________________________________________________________________
leaky_re_lu_46 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_65 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_47 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 257       
Total params: 6,423,553
Trainable params: 6,423,553
Non-trainable params: 0
___________________________________________

(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-154.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-155.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-156.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-157.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-158.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-159.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-16.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-160.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-161.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-162.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-163.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2

(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-24.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-240.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-241.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-242.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-243.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-244.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-245.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-246.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-247.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-248.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-249.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2

(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-325.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-326.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-327.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-328.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-329.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-33.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-330.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-331.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-332.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-333.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-334.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2

(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-410.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-411.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-412.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-413.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-414.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-415.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-416.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-417.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-418.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-419.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2/Random_with_angle115-random-scare-42.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/test2

...

16150 [D loss: 0.736539, acc.: 0.00%] [G loss: 0.741230]
16151 [D loss: 0.735875, acc.: 0.00%] [G loss: 0.741332]
16152 [D loss: 0.735946, acc.: 0.00%] [G loss: 0.740328]
16153 [D loss: 0.736055, acc.: 0.00%] [G loss: 0.740938]
16154 [D loss: 0.735244, acc.: 0.00%] [G loss: 0.741475]
16155 [D loss: 0.736497, acc.: 0.00%] [G loss: 0.740029]
16156 [D loss: 0.735461, acc.: 0.00%] [G loss: 0.739890]
16157 [D loss: 0.734948, acc.: 0.00%] [G loss: 0.740513]
16158 [D loss: 0.735358, acc.: 0.00%] [G loss: 0.741312]
16159 [D loss: 0.734993, acc.: 0.00%] [G loss: 0.741323]
16160 [D loss: 0.734712, acc.: 0.00%] [G loss: 0.741607]
16161 [D loss: 0.734942, acc.: 0.00%] [G loss: 0.740886]
16162 [D loss: 0.735244, acc.: 0.00%] [G loss: 0.740140]
16163 [D loss: 0.734277, acc.: 0.00%] [G loss: 0.739738]
16164 [D loss: 0.734296, acc.: 0.00%] [G loss: 0.739342]
16165 [D loss: 0.734562, acc.: 0.00%] [G loss: 0.739363]
16166 [D loss: 0.734081, acc.: 0.00%] [G loss: 0.739215]
16167 [D loss: 0.733699, acc.: 

16294 [D loss: 0.721157, acc.: 0.00%] [G loss: 0.729858]
16295 [D loss: 0.721348, acc.: 0.00%] [G loss: 0.729438]
16296 [D loss: 0.721073, acc.: 0.00%] [G loss: 0.729522]
16297 [D loss: 0.720881, acc.: 0.00%] [G loss: 0.729469]
16298 [D loss: 0.720814, acc.: 0.00%] [G loss: 0.729328]
16299 [D loss: 0.720781, acc.: 0.00%] [G loss: 0.728963]
16300 [D loss: 0.720465, acc.: 0.00%] [G loss: 0.728956]
16301 [D loss: 0.720940, acc.: 0.00%] [G loss: 0.728865]
16302 [D loss: 0.720550, acc.: 0.00%] [G loss: 0.728652]
16303 [D loss: 0.720493, acc.: 0.00%] [G loss: 0.728454]
16304 [D loss: 0.720245, acc.: 0.00%] [G loss: 0.728244]
16305 [D loss: 0.720108, acc.: 0.00%] [G loss: 0.727920]
16306 [D loss: 0.720104, acc.: 0.00%] [G loss: 0.727587]
16307 [D loss: 0.719781, acc.: 0.00%] [G loss: 0.727791]
16308 [D loss: 0.718945, acc.: 0.00%] [G loss: 0.735083]
16309 [D loss: 0.722002, acc.: 0.00%] [G loss: 0.731923]
16310 [D loss: 0.720943, acc.: 0.00%] [G loss: 0.727324]
16311 [D loss: 0.719479, acc.: 

16438 [D loss: 0.713798, acc.: 0.00%] [G loss: 0.725752]
16439 [D loss: 0.713961, acc.: 0.00%] [G loss: 0.725367]
16440 [D loss: 0.713513, acc.: 0.00%] [G loss: 0.725071]
16441 [D loss: 0.713580, acc.: 0.00%] [G loss: 0.724967]
16442 [D loss: 0.713149, acc.: 0.00%] [G loss: 0.724939]
16443 [D loss: 0.712931, acc.: 0.00%] [G loss: 0.724856]
16444 [D loss: 0.713445, acc.: 0.00%] [G loss: 0.724550]
16445 [D loss: 0.713330, acc.: 0.00%] [G loss: 0.724716]
16446 [D loss: 0.713588, acc.: 0.00%] [G loss: 0.724724]
16447 [D loss: 0.713598, acc.: 0.00%] [G loss: 0.725103]
16448 [D loss: 0.713136, acc.: 0.00%] [G loss: 0.724959]
16449 [D loss: 0.712727, acc.: 0.00%] [G loss: 0.728425]
16450 [D loss: 0.714563, acc.: 0.00%] [G loss: 0.726132]
16451 [D loss: 0.712678, acc.: 0.00%] [G loss: 0.727222]
16452 [D loss: 0.713221, acc.: 0.00%] [G loss: 0.725889]
16453 [D loss: 0.713390, acc.: 0.00%] [G loss: 0.725790]
16454 [D loss: 0.712889, acc.: 0.00%] [G loss: 0.725687]
16455 [D loss: 0.713177, acc.: 

16582 [D loss: 0.710988, acc.: 0.00%] [G loss: 0.724456]
16583 [D loss: 0.710986, acc.: 0.00%] [G loss: 0.724977]
16584 [D loss: 0.711014, acc.: 0.00%] [G loss: 0.724849]
16585 [D loss: 0.711335, acc.: 0.00%] [G loss: 0.724758]
16586 [D loss: 0.711286, acc.: 0.00%] [G loss: 0.724974]
16587 [D loss: 0.711095, acc.: 0.00%] [G loss: 0.725085]
16588 [D loss: 0.710651, acc.: 0.00%] [G loss: 0.725123]
16589 [D loss: 0.711002, acc.: 0.00%] [G loss: 0.724714]
16590 [D loss: 0.711115, acc.: 0.00%] [G loss: 0.724895]
16591 [D loss: 0.710867, acc.: 0.00%] [G loss: 0.724359]
16592 [D loss: 0.710663, acc.: 0.00%] [G loss: 0.724346]
16593 [D loss: 0.710865, acc.: 0.00%] [G loss: 0.724319]
16594 [D loss: 0.710960, acc.: 0.00%] [G loss: 0.724313]
16595 [D loss: 0.710722, acc.: 0.00%] [G loss: 0.727987]
16596 [D loss: 0.715160, acc.: 0.00%] [G loss: 0.973782]
16597 [D loss: 0.742587, acc.: 50.00%] [G loss: 39.823738]
16598 [D loss: 48.781479, acc.: 0.00%] [G loss: 58.818810]
16599 [D loss: 32.578644, a

16726 [D loss: 0.841796, acc.: 0.00%] [G loss: 0.843204]
16727 [D loss: 0.841988, acc.: 0.00%] [G loss: 0.842377]
16728 [D loss: 0.840137, acc.: 0.00%] [G loss: 0.841068]
16729 [D loss: 0.840117, acc.: 0.00%] [G loss: 0.841252]
16730 [D loss: 0.840048, acc.: 0.00%] [G loss: 0.840250]
16731 [D loss: 0.839006, acc.: 0.00%] [G loss: 0.838765]
16732 [D loss: 0.837641, acc.: 0.00%] [G loss: 0.837645]
16733 [D loss: 0.836805, acc.: 0.00%] [G loss: 0.837533]
16734 [D loss: 0.835409, acc.: 0.00%] [G loss: 0.836706]
16735 [D loss: 0.835455, acc.: 0.00%] [G loss: 0.837777]
16736 [D loss: 0.835931, acc.: 0.00%] [G loss: 0.833598]
16737 [D loss: 0.834258, acc.: 0.00%] [G loss: 0.834140]
16738 [D loss: 0.833492, acc.: 0.00%] [G loss: 0.832763]
16739 [D loss: 0.832534, acc.: 0.00%] [G loss: 0.831406]
16740 [D loss: 0.831268, acc.: 0.00%] [G loss: 0.830581]
16741 [D loss: 0.831333, acc.: 0.00%] [G loss: 0.830565]
16742 [D loss: 0.830785, acc.: 0.00%] [G loss: 0.830027]
16743 [D loss: 0.829774, acc.: 

IndentationError: expected an indented block (<ipython-input-31-e6e92e1d671c>, line 74)

In [2]:
import os
import numpy as np
import keras
import matplotlib.pyplot as plt
from tqdm import tqdm

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers
import tensorflow as tf
from keras.layers import Input, Flatten, Reshape
from keras.layers.normalization import BatchNormalization
from PIL import Image

Using TensorFlow backend.
/Users/yiwen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/yiwen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/yiwen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/yiwen/opt/anaconda3/lib/python3.7/sit

In [6]:
(X_train, _), (_, _) = mnist.load_data()

In [7]:
print(np.shape(X_train[0]))


(28, 28)


/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-276.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-277.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-278.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-279.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-280.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-281.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle115-random-scare-282.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle170-random-scare-276.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle170-random-scare-277.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle170-random-scare-278.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle170-random-scare-279.png
(96, 128)
/Users/yiwen/Desktop/GAN/gan/img/Random_with_angle170-random-scar

In [38]:
os.mkdir(os.getcwd() + "/gan/test2/thumbnail")